In [78]:
using PauliPropagation
using Random
Random.seed!(42)

TaskLocalRNG()

### Functions for QAOA on Ising spin chain
- TFIM with obc
- QAOA circuit
- Parameter assignment function

In [79]:
#QAOA circuit
function tfimqaoacircuit(nqubits::Integer, nlayers::Integer; topology=nothing)
    circuit::Vector{Gate} = []

    if isnothing(topology)
        topology = bricklayertopology(nqubits;periodic=false) #chain with obc
    end

    for i in 1:nlayers
        rzzlayer!(circuit, topology)
        for qubit in 1:nqubits
            push!(circuit, CliffordGate(:S, qubit))
            push!(circuit, PauliRotation(:X, qubit, π/2)) # Fig7: P=R_X(π/2)
            push!(circuit, PauliRotation(:Z, qubit))
            push!(circuit, PauliRotation(:X, qubit, π/2))
            push!(circuit, CliffordGate(:S, qubit))

        end
    end
    return circuit
end

tfimqaoacircuit (generic function with 1 method)

In [80]:
#grouped assignment of the params for QAOA circuit
function assign_qaoa_parameters(circuit, nqubits::Integer, nlayers::Integer)
    
    rzz_indices = getparameterindices(circuit, PauliRotation, [:Z, :Z])
    rz_indices = getparameterindices(circuit, PauliRotation, [:Z])

    #group sizes
    Q_minus_1 = (nqubits - 1) * nlayers  # RZZ parameters
    Q = nqubits * nlayers                # RZ parameters

    #group the random values according to circuit layout
    thetas = zeros(maximum(vcat(rzz_indices, rz_indices)))
    thetas[rzz_indices] .= repeat(randn(nlayers), inner=(Q_minus_1 ÷ nlayers))
    thetas[rz_indices] .= repeat(randn(nlayers), inner=(Q ÷ nlayers))

    return thetas
end

assign_qaoa_parameters (generic function with 1 method)

### Initialize circuit
- initial parameters
- define truncations dpending on the method of gradient computation
- for `ReverseDiff`, better to use `max_weight` (number of non-Id in string), `max_freq`, and `max_sins`(number of sins - as in SPD) (see notebook 8)
- define H within loss function

In [92]:
const nq = 16 #we set const for global vars in fcts to opt performance
const nl = 2 #number of layers
const topology = bricklayertopology(nq;periodic=false) #chain with obc  
const g = 2 #paramagnetic phase

2

In [93]:
const circuit = tfimqaoacircuit(nq, nl; topology=topology)
nparams = countparameters(circuit)
println("Number of parameters: ", nparams) 
#generate random parameters
thetas = assign_qaoa_parameters(circuit, nq, nl)
println("Parameters: ", thetas)

Number of parameters: 62
Parameters: [-0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.7021951987576804, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.9129233863399265, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.07258186804586991, -0.

In [94]:
#define the truncations
const max_freq = 30
const max_weight = 5 #number of non-Id Paulis in string

5

In [95]:
#one expectation value (check everything works so far)
@time psum = propagate(circuit, wrapped_H, thetas; max_freq, max_weight);
overlapwithzero(psum)

  0.000272 seconds (792 allocations: 45.297 KiB)


2.4507500860863285

### Loss function and TFIM

In [96]:
function lossfunction(thetas)
    coefftype = eltype(thetas)

    H = PauliSum(nq, coefftype)
    for qind in 1:nq
        add!(H, :X, qind, coefftype(-g))
    end
    for pair in topology
        add!(H, [:Z, :Z], collect(pair), coefftype(1.0))
    end
    
    wrapped_H = wrapcoefficients(H, PauliFreqTracker) #wrap to keep track of path splitting

    # be also need to run the in-place version with `!`, because by default we copy the Pauli sum
    wrapped_H = propagate!(circuit, wrapped_H, thetas; max_freq, max_weight);
    return overlapwithzero(wrapped_H)
end

lossfunction (generic function with 1 method)

In [97]:
@time lossfunction(thetas)

  0.618803 seconds (771.99 k allocations: 38.177 MiB, 3.83% gc time, 99.79% compilation time)


11.658832118986854

In [98]:
using ReverseDiff: GradientTape, gradient!, compile

In [99]:
### This is following an ReverseDiff.jl example

# some inputs and work buffer to play around with
grad_array = similar(thetas);

# pre-record a GradientTape for `gradsimulation` using inputs of length m with Float64 elements
@time const simulation_tape = GradientTape(lossfunction, thetas)

# first evaluation compiles and is slower
@time gradient!(grad_array, simulation_tape, thetas)
# second evaluation
@time gradient!(grad_array, simulation_tape, thetas);

  0.770832 seconds (852.77 k allocations: 40.919 MiB, 2.07% gc time, 98.98% compilation time: 55% of which was recompilation)
  0.003403 seconds


  0.003403 seconds


In [100]:
# gradient descent
eta = 0.1
for i in 1:100
    gradient!(grad_array, simulation_tape, thetas)
    thetas .-= eta * grad_array
    println("Loss: ", lossfunction(thetas))
end

Loss: 4.298454396808317
Loss: -2.721288589809033
Loss: -7.259280371534892
Loss: -9.425226116845376
Loss: -10.354350678623405
Loss: -10.824226799621245
Loss: -11.157401341519547
Loss: -11.46216842216913
Loss: -11.772011543574738
Loss: -12.096103779887097
Loss: -12.435131681982243
Loss: -12.786307045458
Loss: -13.145297961754597
Loss: -13.50730236172797
Loss: -13.867811976524164
Loss: -14.223144390536078
Loss: -14.570730206842688
Loss: -14.909168408074834
Loss: -15.238111052989872
Loss: -15.558067754459717
Loss: -15.870211809472455
Loss: -16.17622514796421
Loss: -16.478164386112837
Loss: -16.77830024215599
Loss: -17.078893367689307
Loss: -17.38190598363639
Loss: -17.68868227111924
Loss: -17.999647165136395
Loss: -18.314079101844253
Loss: -18.630016092244155
Loss: -18.944350930549902
Loss: -19.253144608995253
Loss: -19.55213050871184
Loss: -19.837313880996955
Loss: -20.105528380952407
Loss: -20.354822834572865
Loss: -20.58461359100211
Loss: -20.79561671052118
Loss: -20.989631194474434
Los